In [17]:
import json
import pandas as pd
import numpy as np
import glob
import os

In [18]:
PIXELS_X = 910
PIXELS_Y = 910  # equal to the number of scan lines

In [19]:
THRESHOLD = '0_2'

In [20]:
BASE_NAME = 'yolov3-tiny_3l-tile-34-predictions-thresh-{}.json'.format(THRESHOLD)
PREDICTIONS_DIR = '/Users/darylwilding-mcbride/Downloads/experiments/dwm-test/predictions'
ANNOTATIONS_DIR = '/Users/darylwilding-mcbride/Downloads/experiments/dwm-test/annotations'

In [21]:
predictions_file = '{}/{}'.format(PREDICTIONS_DIR, BASE_NAME)

In [22]:
with open(predictions_file) as file:
    predictions = json.load(file)

In [23]:
len(predictions)

1120

In [24]:
predictions[0]

{'frame_id': 1,
 'filename': '/data/experiments/dwm-test/tiles/190719_Hela_Ecoli_1to1_01/tile-34/frame-1889-tile-34-mz-712-730.png',
 'objects': [{'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.41661,
    'center_y': 0.740934,
    'width': 0.058729,
    'height': 0.046296},
   'confidence': 0.999009},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.153997,
    'center_y': 0.742844,
    'width': 0.087601,
    'height': 0.036858},
   'confidence': 0.991856},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.719132,
    'center_y': 0.722091,
    'width': 0.106037,
    'height': 0.05502},
   'confidence': 0.987076},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.141885,
    'center_y': 0.807706,
    'width': 0.051959,
    'height': 0.034019},
   'confidence': 0.688047},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.135

In [25]:
predictions[1]

{'frame_id': 2,
 'filename': '/data/experiments/dwm-test/tiles/190719_Hela_Ecoli_1to1_01/tile-34/frame-1894-tile-34-mz-712-730.png',
 'objects': [{'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.312258,
    'center_y': 0.73571,
    'width': 0.049204,
    'height': 0.05303},
   'confidence': 0.999926},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.804668,
    'center_y': 0.727952,
    'width': 0.046514,
    'height': 0.055664},
   'confidence': 0.999635},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.242367,
    'center_y': 0.69895,
    'width': 0.078921,
    'height': 0.040997},
   'confidence': 0.997249},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.405212,
    'center_y': 0.795263,
    'width': 0.07493,
    'height': 0.062807},
   'confidence': 0.994766},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.14057

In [26]:
server_url = 'http://spectra-server-lb-1653892276.ap-southeast-2.elb.amazonaws.com/tile/{}/frame/{}'

In [27]:
# for each frame, generate the Via annotations
annotations = {}
for frame in predictions:
    tile_name = os.path.basename(frame['filename'])
    frame_id = int(tile_name.split('-')[1])
    tile_id = int(tile_name.split('-')[3])
    
    entry_name = 'tile-{}-frame-{}'.format(tile_id, frame_id)
    print('entry name: {}'.format(entry_name))
    entry = {}
    entry['file_attributes'] = {}
    entry['filename'] = server_url.format(tile_id, frame_id)
    regions = []
    
    for prediction in frame['objects']:
        class_id = prediction['class_id']
        class_name = prediction['name']
        bounding_box = prediction['relative_coordinates']
        confidence = prediction['confidence']
        
        # calculate the coordinates
        x1 = (bounding_box['center_x'] - (bounding_box['width'] / 2)) * PIXELS_X
        y1 = (bounding_box['center_y'] - (bounding_box['height'] / 2)) * PIXELS_Y
        w = bounding_box['width'] * PIXELS_X
        h = bounding_box['height'] * PIXELS_Y
        x2 = x1 + w
        y2 = y1 + h
        
        # create the region
        region_attributes = {'charge':'{}+'.format(class_id+2), 'isotopes':'0'}
        shape_attributes = {'name':'rect', 'width': int(w), 'height': int(h), 'x': int(x1), 'y': int(y1)}
        region = {'region_attributes':region_attributes, 'shape_attributes':shape_attributes}

        # add this prediction to the region list
        regions.append(region)

    entry['regions'] = regions
    entry['size'] = -1
    
    annotations[entry_name] = entry


entry name: tile-34-frame-1889
entry name: tile-34-frame-1894
entry name: tile-34-frame-1899
entry name: tile-34-frame-1904
entry name: tile-34-frame-1909
entry name: tile-34-frame-1914
entry name: tile-34-frame-1919
entry name: tile-34-frame-1924
entry name: tile-34-frame-1929
entry name: tile-34-frame-1934
entry name: tile-34-frame-1939
entry name: tile-34-frame-1944
entry name: tile-34-frame-1949
entry name: tile-34-frame-1954
entry name: tile-34-frame-1959
entry name: tile-34-frame-1964
entry name: tile-34-frame-1969
entry name: tile-34-frame-1974
entry name: tile-34-frame-1979
entry name: tile-34-frame-1984
entry name: tile-34-frame-1989
entry name: tile-34-frame-1994
entry name: tile-34-frame-1999
entry name: tile-34-frame-2004
entry name: tile-34-frame-2009
entry name: tile-34-frame-2014
entry name: tile-34-frame-2019
entry name: tile-34-frame-2024
entry name: tile-34-frame-2029
entry name: tile-34-frame-2034
entry name: tile-34-frame-2039
entry name: tile-34-frame-2044
entry na

In [28]:
len(annotations)

1120

In [31]:
annotations_filename = '{}/{}'.format(ANNOTATIONS_DIR, BASE_NAME)

In [32]:
with open(annotations_filename, 'w') as file:
    json.dump(annotations, file)